VI.9.32

| Год         | 1910         | 1920        | 1930        | 1940        | 1950        |
| -------     | ------------ | ----------- | ----------- | ----------- | ----------- |
| Население   | 092 228 496  | 106 021 537 | 123 202 624 | 132 164 569 | 151 325 798 |

| Год         | 1960         | 1970        | 1980        | 1990        | 2000        |
| -------     | ------------ | ----------- | ----------- | ----------- | ----------- |
| Население   | 179 323 175  | 203 211 926 | 226 545 805 | 248 709 873 | 281 421 906 |


In [34]:
import numpy as np

years = np.array([1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000])
population = np.array([92228496, 106021537, 123202624, 132164569, 151325798, 179323175, 203211926, 226545805, 248709873, 281421906])

а) Построить интерполянт в форме Ньютона. Вычислить экстраполированное значение численности населения в 2010 году и сравнить с точным значением 308 745 538.

Функция для вычисления разделённой разности:

In [35]:
def sep_diff(years, population):
  sz = np.shape(years)[0]
  sp_tbl = np.zeros([sz, sz])
  for i in range (0, sz):
    sp_tbl[i, 0] = population[i]
  for j in range (1, sz):
    for i in range (1, sz):
      sp_tbl[i, j] = (sp_tbl[i, j - 1] - sp_tbl[i-1, j-1]) / (years[i] - years[i - j])
  
  res = np.zeros([sz])
  for i in range(0, sz):
    res[i] = sp_tbl[i, i]
  
  return res

Функция для вычисления полинома Ньютона. Общий вид формулы выглядит так:

$$N_n(x) = b_0 + b_1\cdot(x-x_0) + b_2\cdot(x-x_0)\cdot(x-x_1) + ... + b_n\cdot(x-x_0)\cdot(x-x_1)\cdot...\cdot(x-x_n)$$

In [36]:
def poly_Newton(years, population, x):
  sp_tbl = sep_diff(years, population)
  sz = np.shape(sp_tbl)[0]
  aux_poly_val = 1
  res = sp_tbl[0]
  for i in range (sz - 1):
    aux_poly_val = aux_poly_val * (x - years[i])
    res = res + sp_tbl[i + 1] * aux_poly_val

  return res

Вычисление значения в 2010 году:

In [37]:
res = poly_Newton(years, population, 2010)
expected_res = 308745538
error = int(np.abs(res / expected_res - 1) * 100)

print("Полученное значение полинома Ньютона:", int(res))
print("Ожидаемое значение полинома Ньютона: ", expected_res)
print("Погрешность вычисления:", error, "%")

Полученное значение полинома Ньютона: 827906509
Ожидаемое значение полинома Ньютона:  308745538
Погрешность вычисления: 168 %


б) Построить сплайн-аппроксимацию, экстраполировать данные на 2010 год.

Используем сплайн:

$$S_k = a_k + b_k (x-x_k) + \frac{c_k}{2} (x-x_k)^2 + \frac{d_k}{6} (x-x_k)^3$$

Также поставим дополнительные условия:

$$f''(1910) = 0 = f''(2000)$$

In [38]:
def calc_diffs(x):
  sz = len(x) - 1
  res = np.zeros(sz)
  for i in range (sz):
    res[i] = x[i+1] - x[i]  
  return res